In [1]:
file = 'C:/Users/tally/Downloads/IPL-2023-schedule.csv'

In [2]:
# load schedule dataset


import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import datetime

from tkinter import *
from tkinter import ttk
from tkcalendar import Calendar
import pandas as pd

from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

from selenium.webdriver.chrome.options import Options

from datetime import datetime
# Get today's date






In [3]:
# drop duplicate rows


def process_Schedule(file):
    schedule_df = pd.read_csv(file)
    schedule_df = schedule_df.drop_duplicates()


    schedule_df["Date"] = pd.to_datetime(schedule_df["Date"], format="%d-%m-%y %H:%M")

    # Convert "Date" column back to string format with "d-m-y" format
    schedule_df["Date"] = schedule_df["Date"].dt.strftime("%d-%m-%Y")

    return schedule_df


In [4]:

def getPointsTable():
    options = Options()
    options.headless = True

    #   create a new Chrome browser instance
    driver = webdriver.Chrome(options=options)
    url = "https://www.iplt20.com/points-table/2023"
    driver.get(url)

    # wait for the dynamic content to load
    driver.implicitly_wait(10)

    # extract the HTML content of the page
    html = driver.page_source

    # create a BeautifulSoup object from the HTML content
    soup = BeautifulSoup(html, 'html.parser')

    # find the table containing the points data
    points_table = soup.find('tbody', {'id': 'pointsdata'})

    # find all rows in the table
    rows = points_table.find_all('tr', {'class': 'ng-scope'})

    data = []

    # extract data from each row
    for row in rows:
        # extract the team name
        team_name = row.find('h2', {'class': 'ih-pt-cont'}).text

        Position = row.find_all('td', {'class': 'ng-binding'})[0].text
        
        # extract the number of matches played
        matches_played = row.find_all('td', {'class': 'ng-binding'})[1].text
        
        # extract the number of points
        Wins = row.find_all('td', {'class': 'ng-binding'})[2].text

        Losses = row.find_all('td', {'class': 'ng-binding'})[3].text

        No_Result = row.find_all('td', {'class': 'ng-binding'})[4].text

        Net_Run_Rate = row.find_all('td', {'class': 'ng-binding'})[5].text

        For = row.find_all('td', {'class': 'ng-binding'})[6].text

        Against = row.find_all('td', {'class': 'ng-binding'})[7].text

        Points = row.find_all('td', {'class': 'ng-binding'})[8].text

        data.append([team_name, matches_played, Wins, Losses, No_Result, Net_Run_Rate, For, Against, Points])

    # create a DataFrame from the extracted data and set the column names
    df = pd.DataFrame(data, columns=['Team', 'Matches Played', 'Wins', 'Losses', 'No Result', 'Net Run Rate', 'For', 'Against', 'Points'])

    df['Matches Played'] = df['Matches Played'].astype(int)
    df['Wins'] = df['Wins'].astype(int)
    df['Losses'] = df['Losses'].astype(int)
    df['No Result'] = df['No Result'].astype(int)
    df['Net Run Rate'] = df['Net Run Rate'].astype(float)
    df['Points'] = df['Points'].astype(int)

    team_name_map = {
    'GT': 'Gujarat Titans',
    'LSG': 'Lucknow Super Giants',
    'CSK': 'Chennai Super Kings',
    'RR': 'Rajasthan Royals',
    'RCB': 'Royal Challengers Bangalore',
    'MI': 'Mumbai Indians',
    'PBKS': 'Punjab Kings',
    'KKR': 'Kolkata Knight Riders',
    'SRH': 'Sunrisers Hyderabad',
    'DC': 'Delhi Capitals'
}

# Replace team names in points_table
    df['Team'] = df['Team'].map(team_name_map)


    return df
    


In [5]:
def calculate_sos(ipl_points_table, team_name, num_matches=2):
    # Get the current position of the team
    team_pos = ipl_points_table[ipl_points_table['Team'] == team_name].index.values[0]
    
    # Select the upcoming matches for the team
    team_matches = ipl_points_table.iloc[team_pos]['Matches']
    upcoming_matches = ipl_points_table.iloc[team_pos:team_pos+team_matches+num_matches]['Team']
    
    # Calculate the strength of schedule
    sos = 0
    for i, opp_name in enumerate(upcoming_matches):
        opp_pos = ipl_points_table[ipl_points_table['Team'] == opp_name].index.values[0]
        opp_points = int(ipl_points_table.iloc[opp_pos]['Points'])
        sos += (i+1) * opp_points
    
    # Return the strength of schedule
    return sos

def calculate_sos_NRR(points_table, upcoming_matches):
    sos = {}
    
    # Calculate SOS using past matches
    for team1 in points_table['Team']:
        opp_win_pct = 0
        for index, row in points_table.iterrows():
            if row['Team'] != team1:
                opp_win_pct += (row['Points'] / (2 * (row['Matches Played'])))
        sos[team1] = opp_win_pct / len(points_table.index)
        sos[team1] += points_table.loc[points_table['Team'] == team1]['Net Run Rate'].values[0]
    
    # Calculate SOS using upcoming matches
    for index, row in upcoming_matches.iterrows():
        home_team = row['Home Team']
        away_team = row['Away Team']
        home_team_sos = sos[home_team]
        away_team_sos = sos[away_team]
        
        # Update SOS based on upcoming match
        sos[home_team] = ((len(points_table.index) + 1) * home_team_sos - away_team_sos) / len(points_table.index)
        sos[away_team] = ((len(points_table.index) + 1) * away_team_sos - home_team_sos) / len(points_table.index)
    
    return sos





In [6]:


def SOS_processing(points_df,schedule_df):
    today = datetime.today().strftime('%d-%m-%Y')
    today_datetime = datetime.strptime(today, '%d-%m-%Y')

    schedule_df['Date'] = pd.to_datetime(schedule_df['Date'], format='%d-%m-%Y')

    upcoming_matches_df = schedule_df[schedule_df['Date'] < today_datetime]

    sos_dict = calculate_sos_NRR(points_df,upcoming_matches_df)

    sos_df = pd.DataFrame.from_dict(sos_dict, orient='index', columns=['Strength Of Schedule'])
    sos_df.index.name = 'Team'
    sos_df = sos_df.reset_index().rename(columns={'index': 'Team'})

    return sos_df,sos_dict

In [7]:
schedule_df = process_Schedule(file)

points_df = getPointsTable()
sos_df,sos_dict = SOS_processing(points_df,schedule_df)



C:\Users\tally\AppData\Local\Temp\ipykernel_12016\2771780469.py:3: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


In [8]:
points_df

,Team,Matches Played,Wins,Losses,No Result,Net Run Rate,For,Against,Points
0,Gujarat Titans,10,7,3,0,0.752,1646/189.0,1586/199.2,14
1,Lucknow Super Giants,10,5,4,1,0.639,1544/176.0,1460/179.3,11
2,Chennai Super Kings,10,5,4,1,0.329,1695/176.5,1660/179.2,11
3,Rajasthan Royals,10,5,5,0,0.448,1806/199.2,1665/193.2,10
4,Royal Challengers Bangalore,9,5,4,0,-0.030,1567/176.2,1605/180.0,10
5,Mumbai Indians,9,5,4,0,-0.373,1662/176.0,1713/174.3,10
6,Punjab Kings,10,5,5,0,-0.472,1787/195.3,1844/191.5,10
7,Kolkata Knight Riders,10,4,6,0,-0.103,1810/196.0,1782/190.5,8
8,Sunrisers Hyderabad,9,3,6,0,-0.540,1437/177.1,1511/174.4,6
9,Delhi Capitals,9,3,6,0,-0.768,1360/179.2,1488/178.1,6


In [9]:
sos_dict

{'Gujarat Titans': 2.3687478567160865,
 'Lucknow Super Giants': 2.2406654140435487,
 'Chennai Super Kings': 0.8754747101856285,
 'Rajasthan Royals': 1.4243867732792528,
 'Royal Challengers Bangalore': 0.3306243356880554,
 'Mumbai Indians': -0.0854673863767312,
 'Punjab Kings': -1.1607483931304914,
 'Kolkata Knight Riders': 0.4262027828637501,
 'Sunrisers Hyderabad': -0.5037923000845834,
 'Delhi Capitals': -1.5540937931845167}

In [10]:
sos_df

,Team,Strength Of Schedule
0,Gujarat Titans,2.368748
1,Lucknow Super Giants,2.240665
2,Chennai Super Kings,0.875475
3,Rajasthan Royals,1.424387
4,Royal Challengers Bangalore,0.330624
5,Mumbai Indians,-0.085467
6,Punjab Kings,-1.160748
7,Kolkata Knight Riders,0.426203
8,Sunrisers Hyderabad,-0.503792
9,Delhi Capitals,-1.554094
